In [2]:
import torch

print(torch.cuda.is_available())

import numpy as np
from scipy.io import loadmat
import h5py

stim_dat=loadmat('data_full.mat')['data']['stim_full'][0][0]
beh_dat=loadmat('data_full.mat')['data']['Behavior_full'][0][0]
hf = h5py.File('TimeSeries.h5', 'r')
neural_dat=np.array(hf['CellResp'])

ModuleNotFoundError: No module named 'torch._C'

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt

import os, datetime
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from torch import nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if cuda else 'cpu')

class SparseAutoencoderL1(nn.Module):
    def __init__(self):
        super(SparseAutoencoderL1, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(83205, 2000),
            nn.ReLU(inplace=True),
            nn.Linear(2000, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 2),
            nn.ReLU(inplace=True)
        )

        self.decoder = nn.Sequential(
            nn.Linear(2, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, 2000),
            nn.ReLU(inplace=True),
            nn.Linear(2000, 83205),
            #nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

def sparse_loss(autoencoder, images):
    loss = 0
    values = images
    for i in range(4):
        fc_layer = list(autoencoder.encoder.children())[2 * i]
        relu = list(autoencoder.encoder.children())[2 * i + 1]
        values = relu(fc_layer(values))
        loss += torch.mean(torch.abs(values))
    for i in range(3):
        fc_layer = list(autoencoder.decoder.children())[2 * i]
        relu = list(autoencoder.decoder.children())[2 * i + 1]
        values = relu(fc_layer(values))
        loss += torch.mean(torch.abs(values))
    return loss

def model_training(autoencoder, train_loader, epoch):
    loss_metric = nn.MSELoss()
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

    autoencoder.train()
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        images = data
        images = Variable(images)
        images = images.view(images.size(0), -1)
        if cuda: images = images.to(device)
        outputs = autoencoder(images)
        mse_loss = loss_metric(outputs, images)
        l1_loss = sparse_loss(autoencoder, images)
        loss = mse_loss + SPARSE_REG * l1_loss
        loss.backward()
        optimizer.step()
        if (i + 1) % LOG_INTERVAL == 0:
            print('Epoch [{}/{}] - Iter[{}/{}], Total loss:{:.4f}, MSE loss:{:.4f}, Sparse loss:{:.4f}'.format(
                epoch + 1, EPOCHS, i + 1, len(train_loader.dataset) // BATCH_SIZE, loss.item(), mse_loss.item(), l1_loss.item()
            ))

def evaluation(autoencoder, test_loader):
    total_loss = 0
    loss_metric = nn.MSELoss()
    autoencoder.eval()
    for i, data in enumerate(test_loader):
        images = data
        images = Variable(images)
        images = images.view(images.size(0), -1)
        if cuda: images = images.to(device)
        outputs = autoencoder(images)
        loss = loss_metric(outputs, images)
        total_loss += loss * len(images)
    avg_loss = total_loss / len(test_loader.dataset)

    print('\nAverage MSE Loss on Test set: {:.4f}'.format(avg_loss))

    global BEST_VAL
    if TRAIN_SCRATCH and avg_loss < BEST_VAL:
        BEST_VAL = avg_loss
        #torch.save(autoencoder.state_dict(), './history/sparse_autoencoder_l1.pt')
        print('Save Best Model in HISTORY\n')


if __name__ == '__main__':

    EPOCHS = 10
    BATCH_SIZE = 128
    LEARNING_RATE = 1e-3
    WEIGHT_DECAY = 1e-5
    LOG_INTERVAL = 100
    SPARSE_REG = 1e-3
    TRAIN_SCRATCH = True        # whether to train a model from scratch
    BEST_VAL = float('inf')
    
    autoencoder = SparseAutoencoderL1()
    if cuda: autoencoder.to(device)
    x_train,x_test,stim_train,stim_test=train_test_split(neural_dat,stim_dat.flatten(), test_size=0.33, random_state=42)
    train_loader = DataLoader(x_train, batch_size=4,
                        shuffle=True, num_workers=4)
    test_loader = DataLoader(x_test, batch_size=1,
                        shuffle=False, num_workers=4)
    if TRAIN_SCRATCH:
        # Training autoencoder from scratch
        for epoch in range(EPOCHS):
            print('started')
            starttime = datetime.datetime.now()
            model_training(autoencoder, train_loader, epoch)
            endtime = datetime.datetime.now()
            print(f'Train a epoch in {(endtime - starttime).seconds} seconds')
            
            # evaluate on test set and save best model
            evaluation(autoencoder, test_loader)
            #utput=autoencoder(neural_dat.to_device(device))
        print('Trainig Complete with best validation loss {:.4f}'.format(BEST_VAL))